<a href="https://colab.research.google.com/github/kah2603/novo-mundo-saude/blob/main/C%C3%B3pia_de_Aula_2_Pr%C3%A1tica_de_Similaridade%2C_Classifica%C3%A7%C3%A3o_e_Sentimentos_Resolvido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 647 kB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=4211b26783e85da1868baa62b8d285113140262ca570ab3fbab568e5dc700680
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading 

In [ ]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  app.launch_new_instance()


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [ ]:
#opção 1 -> montar o drive no colab e acessar o arquivo de embedding do drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
# carregar
##na variável path coloque o caminho do embedding baixado:
path = "/content/drive/MyDrive/files/ptwiki_20180420_100d.txt.bz2"
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

CPU times: user 2min 9s, sys: 1.23 s, total: 2min 11s
Wall time: 2min 14s


In [ ]:
word_vectors.wv["nlp"]

array([ 0.8528,  0.1094, -0.0296, -0.0458, -0.2029, -0.1839, -0.2012,
       -0.111 , -0.0315,  0.1895, -0.2194, -0.3496, -0.1319, -0.1594,
        0.1549, -0.493 ,  0.1897, -0.296 ,  0.0026,  0.2015,  0.0215,
        0.2946, -0.2024,  0.3931,  0.143 ,  0.0667, -0.4868,  0.048 ,
       -0.1028,  0.2372,  0.2042, -0.8025, -0.1221,  0.475 , -0.2997,
        0.219 ,  0.1232, -0.2118,  0.5402, -0.0768,  0.2253,  0.1285,
       -0.6157,  0.1777,  0.0306, -0.1094, -0.1649, -0.386 , -0.2692,
        0.3149,  0.0391, -0.0821, -0.2238,  0.033 ,  0.5032,  0.0836,
        0.1834, -0.2009, -0.0722, -0.1744,  0.3946, -0.0792, -0.0863,
        0.3813,  0.5876, -0.4447, -0.5435,  0.1528, -0.1012, -0.0967,
       -0.2621,  0.4737,  0.3235, -1.1013, -0.0789,  0.1945, -0.1304,
       -0.0379,  0.899 ,  0.0398,  0.2908, -0.4276, -0.0641, -0.3841,
        0.354 ,  0.4139,  0.1076, -0.3439, -0.1474, -0.1215,  0.4359,
       -0.1563,  0.2577,  0.4333,  0.2003, -0.4064, -0.818 ,  0.0812,
       -0.2554, -0.4

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [ ]:
def pre_processamento_texto(corpus):
    #print("#tokenizacao")
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #lowcase 
    corpus_alt = [t.lower() for t in corpus_alt]
    #print("#remove stopwords")
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    #print("#remove pontuação")
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]    
    return corpus_alt

In [ ]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [ ]:
frase1_pre_processada  = pre_processamento_texto(frase1)
frase2_pre_processada  = pre_processamento_texto(frase2)
frase3_pre_processada  = pre_processamento_texto(frase3)
frase4_pre_processada  = pre_processamento_texto(frase4)

Documento
Documento
Documento
Documento


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

Frases 1 e 3 teve a maior.
Frases 1 e 2 teve a menor.
Não faz muito sentido, porque a frase 1 e 2 são frases que falam bem do produto em vários aspectos e não tem similaridade.

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [ ]:
# Criamos o objeto
vect_bag = CountVectorizer()

In [ ]:
docs_bag = vect_bag.fit_transform([' '.join(frase1_pre_processada), \
                                   ' '.join(frase2_pre_processada), \
                                   ' '.join(frase3_pre_processada), \
                                   ' '.join(frase4_pre_processada)])

In [ ]:
frase1_count_vect = docs_bag.todense()[0]
frase2_count_vect = docs_bag.todense()[1]
frase3_count_vect = docs_bag.todense()[2]
frase4_count_vect = docs_bag.todense()[3]

In [ ]:
print("Frase 1 e 2 =>", distance.cosine(frase1_count_vect, frase2_count_vect))
print("Frase 1 e 3 =>", distance.cosine(frase1_count_vect, frase3_count_vect))
print("Frase 2 e 3 =>", distance.cosine(frase2_count_vect, frase3_count_vect))
print("Frase 1 e 4 =>", distance.cosine(frase1_count_vect, frase4_count_vect))

Frase 1 e 2 => 0.8309691490542968
Frase 1 e 3 => 1.0
Frase 2 e 3 => 1.0
Frase 1 e 4 => 1.0


### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

4) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [ ]:
def calcula_embedding_frase(tokens):
    return np.mean(np.array([word_vectors[t] for t in tokens if t in word_vectors.vocab]), axis=0)

In [ ]:
frase1_embedding = calcula_embedding_frase(frase1_pre_processada)
frase2_embedding = calcula_embedding_frase(frase2_pre_processada)
frase3_embedding = calcula_embedding_frase(frase3_pre_processada)
frase4_embedding = calcula_embedding_frase(frase4_pre_processada)

In [ ]:
print("Frase 1 e 2 =>", distance.cosine(frase1_embedding, frase2_embedding))
print("Frase 1 e 3 =>", distance.cosine(frase1_embedding, frase3_embedding))
print("Frase 2 e 3 =>", distance.cosine(frase2_embedding, frase3_embedding))
print("Frase 1 e 4 =>", distance.cosine(frase1_embedding, frase4_embedding))

Frase 1 e 2 => 0.16096973419189453
Frase 1 e 3 => 0.2234203815460205
Frase 2 e 3 => 0.2526938319206238
Frase 1 e 4 => 0.29043054580688477


<b>Atividade </b>

5) Qual diferença entre as representações do exercício 3 e 4? Qual faz mais sentido? Explique.

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

6) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [ ]:
print("Frase 1 e 2 =>", word_vectors.wmdistance(frase1_pre_processada, frase2_pre_processada))
print("Frase 1 e 3 =>", word_vectors.wmdistance(frase1_pre_processada, frase3_pre_processada))
print("Frase 2 e 3 =>", word_vectors.wmdistance(frase2_pre_processada, frase3_pre_processada))
print("Frase 1 e 4 =>", word_vectors.wmdistance(frase1_pre_processada, frase4_pre_processada))

Frase 1 e 2 => 3.0725697404938526
Frase 1 e 3 => 3.759219170455064
Frase 2 e 3 => 3.9048993635316385
Frase 1 e 4 => 3.851623338336251


<b> Atividade </b>

7) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

8) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [ ]:
path_imdb = "/content/drive/MyDrive/PUC/Pós Graduação/NLP | 2-2021/Aulas/Aula 3/Prática/imdb-reviews-pt-br.csv"
df = pd.read_csv(path_imdb)

In [ ]:
df["sentiment"].value_counts()

## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

9) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [ ]:
target = df['sentiment'].replace(['neg','pos'],[0,1])

### Bag of Words (BOW)

<b> Atividade </b>

10) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_return_str(corpus):
    #print("#tokenizacao")
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #print("#remove stopwords")
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    #print("#remove pontuação")
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    #print("#remove numeros")
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt]
    
    corpus_alt_str = ' '.join(corpus_alt).lower()
    
    return corpus_alt_str

In [ ]:
df["text_pt_sem_stopwords"] = df["text_pt"].progress_apply(lambda x: pre_processamento_texto_return_str(x))

  0%|          | 0/49459 [00:00<?, ?it/s]

<b> Atividade </b>

11) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [ ]:
# Criamos o objeto
vect_bag = CountVectorizer()

In [ ]:
X_bag = vect_bag.fit_transform(df['text_pt_sem_stopwords'])

In [ ]:
vocabulario = vect_bag.get_feature_names()

In [ ]:
print("Vocabulario",len(vocabulario))
print("Features",X_bag.shape)
print("Target",target.shape)

Vocabulario 127624
Features (49459, 127624)
Target (49459,)


### Embedding

<b> Atividade </b>

12) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_return_token(corpus):
    #print("#tokenizacao")
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #lowcase 
    corpus_alt = [t.lower() for t in corpus_alt]
    #print("#remove stopwords")
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    #print("#remove pontuação")
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    #print("#remove numeros")
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt]
        
    return corpus_alt

In [ ]:
df["text_pt_sem_stopwords_token"] = df["text_pt"].progress_apply(lambda x: pre_processamento_texto_return_token(x))

  0%|          | 0/49459 [00:00<?, ?it/s]


<b> Atividade </b>

13) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [ ]:
def calcula_embedding_frase(tokens):
    return np.mean([word_vectors[t] for t in tokens if t in word_vectors.vocab], axis=0)

In [ ]:
X_embedding = df["text_pt_sem_stopwords_token"].progress_apply(lambda x: calcula_embedding_frase(x))

  0%|          | 0/49459 [00:00<?, ?it/s]

## Treinamento

### CountVectorizer


<b> Atividade </b>

14) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)


<b> Atividade </b>

15) Treine com uma regressão logística

In [ ]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow,y_train_bow)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


<b> Atividade </b>

16) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [ ]:
y_pred = modelo_bow.predict(X_test_bow)

In [ ]:
print(classification_report(y_test_bow, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

17) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [ ]:
X_train_embedding, X_test_embedding, y_train_embedding, y_test_embedding = \
train_test_split(X_embedding.values, target,random_state=123)

In [ ]:
X_train_embedding = pd.DataFrame([x for x in X_train_embedding])
#X_train_embedding = np.column_stack(X_train_embedding).T.shape
X_train_embedding.shape

(37094, 100)

In [ ]:
X_test_embedding = pd.DataFrame([x for x in X_test_embedding])
#X_test_embedding = np.column_stack(X_test_embedding).T.shape
X_test_embedding.shape

(12365, 100)


<b> Atividade </b>

18) Treine com uma regressão logística

In [ ]:
modelo_embedding = LogisticRegression()
modelo_embedding.fit(X_train_embedding,y_train_embedding)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


<b> Atividade </b>

19) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [ ]:
y_pred = modelo_embedding.predict(X_test_embedding)

In [ ]:
print(classification_report(y_test_embedding, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      6112
           1       0.78      0.77      0.77      6253

    accuracy                           0.77     12365
   macro avg       0.77      0.77      0.77     12365
weighted avg       0.77      0.77      0.77     12365




<b> Atividade </b>

20) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

- No bagofwords as palavras são bem marcadas. Se tenho a palavra triste, o modelo entende o que está palavra representa. Já no embedding o modelo não sabe a representação de triste, ele sabe a representação geral.

- O embedding é de apenas 100 dimensões e quando tiramos a média perdemos informações.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

21) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
analyzer.polarity_scores(texto_neg)

{'compound': 0.3958, 'neg': 0.126, 'neu': 0.76, 'pos': 0.114}

In [ ]:
analyzer.polarity_scores(texto_pos)

{'compound': 0.9969, 'neg': 0.084, 'neu': 0.737, 'pos': 0.179}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [ ]:
from textblob import TextBlob

In [ ]:
sentence=TextBlob(texto_neg)
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

In [ ]:
sentence=TextBlob(texto_pos)
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [ ]:
from afinn import Afinn

In [ ]:
afinn = Afinn()

In [ ]:
afinn.score(texto_pos)

55.0

In [ ]:
afinn.score(texto_neg)

8.0

<b> Atividade </b>

24) Para você, qual ferramenta teve melhor comportamento? Justifique com exemplos.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)